<a href="https://colab.research.google.com/github/Sawera557/Upscale_video_with_Gans/blob/main/Real_ESRGAN_Batch_Process_of_Frames_of_video_By_Sawera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Notes

* On free colab, try to use short length videos, it works good!
* Read Instructions


In [2]:
%cd /content
!rm -rf Real-ESRGAN
# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop
# Download the pre-trained model
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models

/content
Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 759, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 759 (delta 0), reused 2 (delta 0), pack-reused 755
Receiving objects: 100% (759/759), 5.39 MiB | 25.54 MiB/s, done.
Resolving deltas: 100% (406/406), done.
/content/Real-ESRGAN
/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_

# 2. Now we create some important folders



*   First Run this cell

*   Once you've run the cell , now we can see that a folder named "videos" created inside the Real-ESRGAN folder


*   Just drag and drop any number of videos to this folder. (videos that you want to upscale)







In [3]:
import os
from google.colab import files
import shutil

upload_folder = 'upload'
result_folder = 'results'
video_folder = 'videos'
video_result_folder = 'results_videos'
video_mp4_result_folder = 'results_mp4_videos'

if os.path.isdir(upload_folder):
  print(upload_folder+" exists")
else :
  os.mkdir(upload_folder)

if os.path.isdir(video_folder):
  print(video_folder+" exists")
else :
  os.mkdir(video_folder)

if os.path.isdir(video_result_folder):
  print(video_result_folder+" exists")
else :
  os.mkdir(video_result_folder)

if os.path.isdir(video_mp4_result_folder):
  print(video_mp4_result_folder+" exists")
else :
  os.mkdir(video_mp4_result_folder)

if os.path.isdir(result_folder):
  print(result_folder+" exists")

else :
  os.mkdir(result_folder)




# 3. Inference



*   This will take a good amount of time depends on the length of your video and how many videos you want to upscale, the shorter the size of video, the less time it takes.

*   Once the upscaling process is complete , you can find your upscaled video results , under the "results_mp4_videos" folder in Real-esrgan.





#### In this code we are extracting the FPS of original video so we get the same improved upscaled video also we are initializing this fps

In [8]:
import cv2

def get_fps(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video file is opened successfully
    if not cap.isOpened():
        print("Error: Couldn't open the video file")
        return None

    # Get the frames per second (fps) of the video
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Release the video capture object
    cap.release()

    return fps

# Path to the video file
video_path = "/content/Real-ESRGAN/videos/Me at the zoo_360p.mp4"

# Call the function to get the fps
fps = get_fps(video_path)
original_FPS_of_Video = fps

if fps is not None:
    print(f"Frames per second (fps) of the video: {fps}")
else:
    print("Failed to get the frames per second (fps) of the video")


Frames per second (fps) of the video: 15.0


Initialze this

##IMPORTANT if you get any error try

Run this code

In [ ]:
!pip install torchvision

###If you still get the error of torchvision

Do Following and rerun inference cell

IN THIS FILE /usr/local/lib/python3.10/dist-packages/basicsr/data/degradations.py

Change this line

from torchvision.transforms.functional_tensor import rgb_to_grayscale

To:

from torchvision.transforms.functional import rgb_to_grayscale

GET FPS of (Frames per second) of video

In [16]:
import cv2
import numpy as np
import glob
from os.path import isfile, join
import subprocess
from IPython.display import clear_output

# assign directory
directory = '/content/Real-ESRGAN/videos' #PATH_WITH_INPUT_VIDEOS
zee = 0

#deletes frames from previous video
for f in os.listdir(upload_folder):
    os.remove(os.path.join(upload_folder, f))

#deletes upscaled frames from previous video
for f in os.listdir(result_folder):
    os.remove(os.path.join(result_folder, f))

#clearing previous .avi files
for f in os.listdir(video_result_folder):
    os.remove(os.path.join(video_result_folder, f))

#clearing .mp4 result files
for f in os.listdir(video_mp4_result_folder):
    os.remove(os.path.join(video_mp4_result_folder, f))


def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    #for sorting the file names properly
    files.sort(key = lambda x: int(x[5:-4]))
    size2 = (0,0)

    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        size2 = size
        print(filename)
        #inserting the frames into an image array
        frame_array.append(img)
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size2)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()


for filename in os.listdir(directory):

    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):


      print("PROCESSING :"+str(f)+"\n")
      # Read the video from specified path

      #video to frames
      cam = cv2.VideoCapture(str(f))

      try:

          # PATH TO STORE VIDEO FRAMES
          if not os.path.exists('/content/Real-ESRGAN/upload'):
              os.makedirs('/content/Real-ESRGAN/upload')

      # if not created then raise error
      except OSError:
          print ('Error: Creating directory of data')

      # frame
      currentframe = 0

      #clear all folders


      #deletes upscaled frames from previous video
      #for f in os.listdir(result_folder):
      #  os.remove(os.path.join(result_folder, f))




      while(True):

          # reading from frame
          ret,frame = cam.read()

          if ret:
              # if video is still left continue creating images
              name = '/content/Real-ESRGAN/upload/frame' + str(currentframe) + '.jpg'

              # writing the extracted images
              cv2.imwrite(name, frame)


                # increasing counter so that it will
                # show how many frames are created
              currentframe += 1
          else:
              #deletes all the videos you uploaded for upscaling
              #for f in os.listdir(video_folder):
              #  os.remove(os.path.join(video_folder, f))

              break

        # Release all space and windows once done
      cam.release()
      cv2.destroyAllWindows()

      #apply super-resolution on all frames of a video
      #scale factor is by 3.5x

      !python inference_realesrgan.py -n RealESRGAN_x4plus -i upload --outscale 2 --face_enhance --tile 128

      #after upscaling just delete the source frames
      for f in os.listdir(upload_folder):
          os.remove(os.path.join(upload_folder, f))

      #rename all frames in "results" to remove the 'out' substring from the processing results
      paths = (os.path.join(root, filename)
              for root, _, filenames in os.walk('/content/Real-ESRGAN/results')
              for filename in filenames)
      for path in paths:
          newname = path.replace('_out', '')
          if newname != path:
              os.rename(path, newname)

      #convert super res frames to .avi
      pathIn = '/content/Real-ESRGAN/results/'

      zee = zee+1
      fName = "video"+str(zee)
      filenameVid = f"{fName}.avi"

      pathOut = "/content/Real-ESRGAN/results_videos/"+filenameVid

      fps = original_FPS_of_Video #change this to FPS of your source video

      convert_frames_to_video(pathIn, pathOut, fps)

      #after processing frames converted to .avi video , delete upscaled frames from previous video
      for f in os.listdir(result_folder):
          os.remove(os.path.join(result_folder, f))

      #convert .avi to .mp4
      src = '/content/Real-ESRGAN/results_videos/'
      dst = '/content/Real-ESRGAN/results_mp4_videos/'

      for root, dirs, filenames in os.walk(src, topdown=False):
          #print(filenames)
          for filename in filenames:
              print('[INFO] 1',filename)
              try:
                  _format = ''
                  if ".flv" in filename.lower():
                      _format=".flv"
                  if ".mp4" in filename.lower():
                      _format=".mp4"
                  if ".avi" in filename.lower():
                      _format=".avi"
                  if ".mov" in filename.lower():
                      _format=".mov"

                  inputfile = os.path.join(root, filename)
                  print('[INFO] 1',inputfile)
                  outputfile = os.path.join(dst, filename.lower().replace(_format, ".mp4"))
                  subprocess.call(['ffmpeg', '-i', inputfile, outputfile])
              except:
                  print("An exception occurred")

      clear_output(wait=True)

      #clearing previous .avi files
      for f in os.listdir(video_result_folder):
          os.remove(os.path.join(video_result_folder, f))

      #deletes frames from previous video
      #for f in os.listdir(upload_folder):
      #  os.remove(os.path.join(upload_folder, f))



      # if it is out of memory, try to use the `--tile` option
# We upsample the image with the scale factor X3.5

# Arguments
# -n, --model_name: Model names
# -i, --input: input folder or image
# --outscale: Output scale, can be arbitrary scale factore.

PROCESSING :/content/Real-ESRGAN/videos/Me at the zoo_360p.mp4

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Testing 0 frame0
	Tile 1/6
	Tile 2/6
	Tile 3/6
	Tile 4/6
	Tile 5/6
	Tile 6/6
Testing 1 frame1
	Tile 1/6
	Tile 2/6
	Tile 3/6
	Tile 4/6
	Tile 5/6
	Tile 6/6
Testing 2 frame10
	Tile 1/6
	Tile 2/6
	Tile 3/6
	Tile 4/6
	Tile 5/6
	Tile 6/6
Testing 3 frame100
	Tile 1/6
	Tile 2/6
	Tile 3/6
	Tile 4/6
	Tile 5/6
	Tile 6/6
Testing 4 frame101
	Tile 1/6
	Tile 2/6
	Tile 3/6
	Tile 4/6
	Tile 5/6
	Tile 6/6
Testing 5 frame102
	Tile 1/6
	Tile 2/

# 4. Download Results



*   All your upscaled .mp4 files will be inside the results_mp4_videos folder (inside Real-ESRGAN folder)


*   You can right click on the needed file and download it from there.






# 5. Run this cell after tyour videos have been upscaled
# (This Deletes processed data)



*   IMPORTANT : this DELETES previous frames and videos


In [17]:


#deletes frames from previous video
for f in os.listdir(upload_folder):
    os.remove(os.path.join(upload_folder, f))

#deletes upscaled frames from previous video
for f in os.listdir(result_folder):
    os.remove(os.path.join(result_folder, f))

#deletes all the videos you uploaded for upscaling
for f in os.listdir(video_folder):
    os.remove(os.path.join(video_folder, f))

#clearing previous .avi files
for f in os.listdir(video_result_folder):
    os.remove(os.path.join(video_result_folder, f))

#clearing .mp4 result files
for f in os.listdir(video_mp4_result_folder):
    os.remove(os.path.join(video_mp4_result_folder, f))